# Stacked Denoising Autoencoder regression

Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import pandas
from keras import backend as K
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras_tqdm import TQDMCallback, TQDMNotebookCallback
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, GaussianNoise, LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4184927994315154264
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4951913267
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16846310861428380862
physical_device_desc: "device: 0, name: GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


Read and load datasets

In [3]:
# load dataset
dataframe = pandas.read_excel("dna_methy.xlsx", delim_whitespace=True, header=None, skiprows=1)
dataset = dataframe.values
print("data shape :", dataframe.shape)
data_len = dataset.shape[1]
print("no. of features : ", data_len )

data shape : (194, 5001)
no. of features :  5001


Splitting the dataset into taining and testing set

In [4]:
# split into input (X) and output (Y) variables
train, test = train_test_split(dataset, test_size=0.2)
trainX = train[:,1:data_len]
#trainY = train[:,data_len-1]
testX = test[:,1:data_len]
#testY = test[:,data_len-1]

print("trainX : ", trainX.shape)
#print("trainY : ", trainY.shape)
print("testX : ", testX.shape)
#print("testY : ", testY.shape)
#np.savetxt('Y.txt', trainY)
#np.savetxt('Y_test.txt', testY)

trainX :  (155, 5000)
testX :  (39, 5000)


Normalizing the data

In [5]:
# Normalizing train data

max_abs_scaler = MaxAbsScaler()
normX = max_abs_scaler.fit_transform(trainX)
input_shape = normX.shape
#print("X norm : ", normX[:,15439])
print("normX shape : ", input_shape)
#np.savetxt('xnorm.txt', normX)
small_train = normX[:, 0:8000]

# Normalizing test data

max_abs_scaler = MaxAbsScaler()
normXt = max_abs_scaler.fit_transform(testX)
#np.savetxt('test_set.txt', normXt)
print("X norm shape :", normXt.shape)
small_test = normXt[:, 0:8000]

print("small train : ", small_train.shape)
print("smnall test : ", small_test.shape)

normX shape :  (155, 5000)
X norm shape : (39, 5000)
small train :  (155, 5000)
smnall test :  (39, 5000)


In [6]:
earlyStopping = EarlyStopping(monitor='loss', patience=50)
checkpointer = ModelCheckpoint(filepath='test.h5', verbose=1, save_best_only=True)
input_dims = data_len-1
lr_decay = ReduceLROnPlateau(monitor='val_loss', mode = 'min',factor=0.5, patience=5, min_lr=0.0001, verbose=0)

# 1 - Simple Denoising Autoencoder

COMPULSORY!!! run the function below before go to the Stacked Denoising Autoencoder

In [7]:
def dae (inputX, input_dims, output_dims, epoch, activation, loss, opti):
    
    #input_dims = inputX.shape
    #print("input dims : ", input_dims)
    #config = tf.ConfigProto()
    #config.gpu_options.per_process_gpu_memory_fraction = 1.0
    #session = tf.Session(config=config)
    
    model = Sequential()

    model.add(Dense(input_dims, input_dim = input_dims))
    model.add(GaussianNoise(0.5))
    model.add(Dense(output_dims, activation= activation, kernel_regularizer = regularizers.l1(0.01)))
    model.add(Dense(input_dims, activation= activation))
    
    model.compile(loss = loss, optimizer = opti)
    model.summary()
    model.fit(inputX, inputX, epochs = epoch, batch_size = 4, callbacks = [checkpointer, lr_decay, TQDMNotebookCallback()], verbose =0)
    
    return model
    

Don't need to run code below if you want to go straight to the Stacked Denoising Autoencoder. This code is to show how the Denoising Autoencoder works.

In [8]:
autoencoder = dae(normX, 
                  input_dims = input_dims, 
                  output_dims = input_dims, 
                  epoch = 3, 
                  activation = 'relu', 
                  loss = 'mse', 
                  opti = 'adam',
                 )

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 5000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
dense_3 (Dense)              (None, 5000)              25005000  
Total params: 75,015,000
Trainable params: 75,015,000
Non-trainable params: 0
_________________________________________________________________


C:\Users\faizm\Anaconda3\envs\faiz-gpu\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\faizm\Anaconda3\envs\faiz-gpu\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


# 2 - Stacked Denoising Autoencoder - pretrain

In [9]:
# Hyper-parameter

layers = [input_dims, 3000, 1000, 500, 300, 100]
epoch = 3
optimizer = 'adamax'
activation = 'relu'
loss = 'mse'

In [10]:
def sdae_pretrain (inputX, layers, activation, epoch, optimizer, loss):
    
    encoder = []
    decoder = []
    ae = []
    
    for i in range(len(layers)-1):
            # Greedily train each layer
            print("Now pretraining layer {} [{}-->{}]".format(i+1, layers[i], layers[i+1]))

            input_dims = layers[i]
            output_dims = layers[i+1]
            
            autoencoder = dae(inputX, input_dims, output_dims, epoch, activation, loss, optimizer)
            
            enc = Sequential()
            enc.add(Dense(output_dims, input_dim=input_dims))
            enc.compile(loss=loss, optimizer=optimizer)
            enc.layers[0].set_weights(autoencoder.layers[2].get_weights())
            inputX = enc.predict(inputX)
            print("check dimension : ", inputX.shape)
            enc.summary()
            #print(i)
            encoder.append(autoencoder.layers[2].get_weights())
            decoder.append(autoencoder.layers[3].get_weights())
            ae.append(autoencoder)

    
    return ae

In [ ]:
test = sdae_pretrain(normX, layers = layers, activation = activation, epoch = epoch, optimizer = optimizer, loss = loss)

Now pretraining layer 1 [5000-->3000]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5000)              25005000  
_________________________________________________________________
gaussian_noise_2 (GaussianNo (None, 5000)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 3000)              15003000  
_________________________________________________________________
dense_6 (Dense)              (None, 5000)              15005000  
Total params: 55,013,000
Trainable params: 55,013,000
Non-trainable params: 0
_________________________________________________________________


C:\Users\faizm\Anaconda3\envs\faiz-gpu\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
C:\Users\faizm\Anaconda3\envs\faiz-gpu\lib\site-packages\keras\callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning



check dimension :  (155, 3000)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 3000)              15003000  
Total params: 15,003,000
Trainable params: 15,003,000
Non-trainable params: 0
_________________________________________________________________
Now pretraining layer 2 [3000-->1000]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 3000)              9003000   
_________________________________________________________________
gaussian_noise_3 (GaussianNo (None, 3000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1000)              3001000   
_________________________________________________________________
dense_10 (Dense)             (None, 3000)              30030


check dimension :  (155, 1000)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 1000)              3001000   
Total params: 3,001,000
Trainable params: 3,001,000
Non-trainable params: 0
_________________________________________________________________
Now pretraining layer 3 [1000-->500]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1000)              1001000   
_________________________________________________________________
gaussian_noise_4 (GaussianNo (None, 1000)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_14 (Dense)             (None, 1000)              501000  

OPTIONAL: Saving the models for each layer

In [ ]:
for i in range(len(test)):
    
    test[i].save("pre_train_l1" + str(i) + ".hd5")

# 3 - Stacked Denoising Autoencoder - fine tuning

In [ ]:
test = []

for i in range (len(layers)-1):
    
    test.append(load_model("pre_train_l1"+ str(i) + ".hd5"))

In [ ]:
len(test)

In [ ]:
def fine_tuning(weights, inputX, inputXt, inputY, inputYt, layers, epoch, activation, batch, optimizer, loss):

    encoder = []
    decoder = []

    for i in range(len(test)):
    
        encoder.append(test[i].layers[2].get_weights())
        decoder.append(test[i].layers[3].get_weights())
    
    
    ft = Sequential()
    ft.add(Dense(layers[0], input_dim=layers[0]))
    ft.add(GaussianNoise(0.5))

    for i in range(len(layers)-1):
        #print(i)
        ft.add(Dense(layers[i+1], activation = activation, weights = encoder[i], kernel_regularizer = regularizers.l1_l2(0.01)))
        
    for i in reversed(range(len(layers)-1)):
        #print(i)
        ft.add(Dense(layers[i], activation = activation, weights = decoder[i]))
    
    #ft.add(Dense(1, kernel_initializer = 'normal'))   
    ft.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    ft.fit(normX, normX, epochs = epoch, batch_size = batch, validation_data=(inputXt, inputXt))
    ft.summary()
    
    return ft

In [ ]:
tryla = fine_tuning(test, normX, normXt, trainY, testY, layers = layers, epoch = 1000, activation = 'relu', batch = 16, optimizer = 'adam', loss = 'mse')

In [ ]:
tryla.predict(x=normXt)

In [ ]:
tryla.save("fine_tune.hdf5")